## Project Name
> Auxilary tools and solution for pre/post
processing of audio and video based on ffmpeg
inspired by [ffmpeg]('https://pypi.org/project/python-ffmpeg-video-streaming/)

# 1. Installing/updates of libraries

In [1]:

# from datetime import datetime
!pip install python-ffmpeg-video-streaming
!apt install ffmpeg
!pip install ffmpy

from IPython.display import HTML
from base64 import b64encode

import ffmpeg_streaming
from ffmpeg_streaming import Formats
import ffmpy
import os
import datetime as dt
import shutil



Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
  Created wheel for ffmpy: filename=ffmpy-0.2.3-cp36-none-any.whl size=4623 sha256=d1b69edd6f3792d38d702d0e1f2a2e27ea51d7d84c484ea020eeaacdc99d1e13
  Stored in directory: /root/.cache/pip/wheels/99/51/33/ca9d92c9fa38a196ad8647f5fcc9eadbca8a96bd5f8a2b5f7d
Successfully built ffmpy


# 2. Mounting google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Standard snipet to prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [3]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# 3. Useful functions

Playing video file in colab window

In [17]:
video_file='/content/drive/MyDrive/ff_temp/204800recod.mp4'
mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.

If it gives error there is need to decode piece of mp4 again like below

In [19]:
input_video='/content/drive/MyDrive/ff_temp/205101.mp4'
output_video='/content/drive/MyDrive/ff_temp/205101recod.mp4'

In [20]:
os.system(f"ffmpeg -i {input_video} -vcodec libx264 {output_video}")

0

Preparing temporary folder for video processing

In [21]:
#preparing temporary folder on google drive for video
def folder_check(path):
    try_num = 1
    oripath = path[:-1] if path.endswith('/') else path
    if(not os.path.exists(path)):
      os.mkdir(path)
      print(path, '   created')
    else:    
      while os.path.exists(path):
          print("Delete content of  existing folder " + path + "?(Y/N)")
          decision = input()
          if decision == "Y":
              shutil.rmtree(path, ignore_errors=True)
              os.mkdir(path)

              break
          else:
            path = oripath + "_%d/"%try_num
            try_num += 1
            if(not os.path.exists(path)):
              print("Create new folder " + path + "?(Y/N)")
              decision = input()
              if decision == "Y":
                os.mkdir(path)
                print(path, 'was created')
                break
    

    
    return path

In [22]:
# generating file name with timestamp info for later comparison
def gen_file_with_timestamp(name='name'):
    timestamp = dt.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".mp4"
    return file_name

saving stream in chunks  taken from https://stackoverflow.com/questions/10588001/how-to-record-udp-stream-with-ffmpeg-in-chunks
after -segment time  is chunk time given in seconds 

In [35]:
#saving stream to chunk file with given timestamp
def save_m3u8(path,  time_chunk):
  print('chunks will be saved in {} directory to stop press stop button on the left'.format(path) )

  os.system(f"ffmpeg -re -i https://tagesschau-lh.akamaihd.net/i/tagesschau_1@119231/master.m3u8 -vcodec libx264 -vb 500000 -g 60 -vprofile main -acodec aac -ab 128000 -ar 48000 -ac 2 -vbsf h264_mp4toannexb -b 1000k -minrate 1000k -maxrate 1000k -strict experimental -f stream_segment -segment_format mpegts -segment_time {time_chunk} -segment_atclocktime 1 -reset_timestamps 1 -strftime 1 {path}%H%M%S.mp4")

In [33]:
path = folder_check('/content/drive/MyDrive/ff_temp/')

Delete content of  existing folder /content/drive/MyDrive/ff_temp/?(Y/N)
N
Delete content of  existing folder /content/drive/MyDrive/ff_temp_1/?(Y/N)
N
Create new folder /content/drive/MyDrive/ff_temp_2/?(Y/N)
Y
/content/drive/MyDrive/ff_temp_2/ was created


As a source of video for later checking  saved m3u8 video stream will be used

In [ ]:
time_chunk = 210 # in seconds
save_m3u8(path,  time_chunk)

chunks will be saved in /content/drive/MyDrive/ff_temp_2/ directory to stop press stop button on the left
